In [ ]:
import os
import xarray as xr
import numpy as np
import glob

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

import lmom as lmom
import MetFunctions as mf
import MetFunctions01 as mf1
from scipy.stats import spearmanr
from scipy.stats import norm

In [ ]:
def AddSavg(am1, S1_savg, S2_savg, C_savg):
    conditionsALL = [am1.season=='DJF', am1.season=='JJA', am1.season=='MAM', am1.season=='SON']

    choicesS1 = [S1_savg['DJF'], S1_savg['JJA'], S1_savg['MAM'], S1_savg['SON']]
    choicesS2 = [S2_savg['DJF'], S2_savg['JJA'], S2_savg['MAM'], S2_savg['SON']]
    choicesC = [C_savg['DJF'], C_savg['JJA'], C_savg['MAM'], C_savg['SON']]

    am1['S1savg'] = np.select(conditionsALL, choicesS1, default='ERROR').astype(float)
    am1['S2savg'] = np.select(conditionsALL, choicesS2, default='ERROR').astype(float)
    am1['Csavg'] = np.select(conditionsALL, choicesC, default='ERROR').astype(float)
    am1['S1/S1savg'] = am1.P_S1rol / am1.S1savg
    am1['S2/S2savg'] = am1.P_S2rol / am1.S2savg
    am1['C/Csavg'] = am1.Conf / am1.Csavg
    return am1

Read each file and get the AM sets per file, combine them per confluence (50,000 peaks) per each set according to the contidion (S1, S2, C), and store the dataframes for later
Loops:

    C - Confluences

    j - files
    
    i - ensembles per file

In [ ]:
# #RAIN 
conf = pd.read_csv('Confluences_wnames.csv', delimiter=';')
w = 5
# c = 0
am1 = pd.DataFrame()
am2 = pd.DataFrame()
am3 = pd.DataFrame()
savg_S1 = 0
savg_S2 = 0
savg_C = 0
count = 0

files = 2
ensemble = 20

for c in range(2): #len(conf)
    sv1 =  r"D:\AMrain\C%d_" %c
    for j in range(files):
        path1 = r"D:\TimeSeries\ds%d" %j
        for i in range(ensemble):
            path2 = r"_%draina.npy" %i
            path = path1 + path2
            ds_rain = np.load(path, allow_pickle=True).item()
            am1_, am2_, am3_, savg_S1_, savg_S2_, savg_C_ = mf1.AM(ds_rain, conf, c, w)
            am1 = pd.concat([am1, am1_])
            am2 = pd.concat([am2, am2_])
            am3 = pd.concat([am3, am3_])
            savg_S1 += savg_S1_
            savg_S2 += savg_S2_
            savg_C += savg_C_
            count += 1
    
    #from all the loops, get am1, am2, am3, savg_S1, savg_S2, savg_C combinado de los 50,000 anos
    S1_savg = savg_S1 / count
    S2_savg = savg_S2 / count
    C_savg = savg_C / count
    
    AM1 = AddSavg(am1, S1_savg, S2_savg, C_savg)
    AM2 = AddSavg(am2, S1_savg, S2_savg, C_savg)
    AM3 = AddSavg(am3, S1_savg, S2_savg, C_savg)
    #now we want to sabe the 3 data frames per confluence
    AM1.to_pickle(sv1 + "AM1.pkl")
    AM2.to_pickle(sv1 + "AM2.pkl")
    AM3.to_pickle(sv1 + "AM3.pkl")


In [ ]:
#RAIN 1 day 
conf = pd.read_csv('Confluences_wnames.csv', delimiter=';')
w = 1 #5 do it with 1 to compare the differeces between 1 and 5 days
# c = 0
am1 = pd.DataFrame()
am2 = pd.DataFrame()
am3 = pd.DataFrame()
savg_S1 = 0
savg_S2 = 0
savg_C = 0
count = 0

files = 2
ensemble = 20


for j in range(23, 25):  # Maybe only 5 files? -->10,000
    path1 = r"F:\TimeSeries\ds%d" %j
    print(f'Starting File {j}')
    for i in range(0, 20):
        print(f'Ensemble {i}')
        path2 = r"_%draina.npy" %i
        path = path1 + path2
        ds_rain = np.load(path, allow_pickle=True).item()
        for c in range(0, len(conf)): #len(conf)
            sv =  r"F:\AMrain1d\C%d_" %c + r"%d_" %j + r"%d" %i 
            am1_, am2_, am3_, savg_S1_, savg_S2_, savg_C_ = mf1.AM(ds_rain, conf, c, w)
            am1_.to_pickle(sv + "AM1.pkl")
            am2_.to_pickle(sv + "AM2.pkl")
            am3_.to_pickle(sv + "AM3.pkl")
            savg_S1_.to_pickle(sv + "savg_S1.pkl")
            savg_S2_.to_pickle(sv + "savg_S2.pkl")
            savg_C_.to_pickle(sv + "savg_C.pkl")


In [ ]:
#SNOW
conf = pd.read_csv('Confluences_wnames.csv', delimiter=';')
w = 5
# c = 0
am1 = pd.DataFrame()
am2 = pd.DataFrame()
am3 = pd.DataFrame()
savg_S1 = 0
savg_S2 = 0
savg_C = 0
count = 0

files = 25
ensemble = 20

for j in range(20, 25):  #10
    print(f'Starting File {j}')
    path1 = r"F:\TimeSeries\ds%d" %j
    for i in range(0, 20): #20
        print(f'Ensemble {i}')
        path2 = r"_%dsnow.npy" %i
        path = path1 + path2
        ds_rain = np.load(path, allow_pickle=True).item()
        for c in range(0, len(conf)): #len(conf)
            sv =  r"F:\AMsnow\C%d_" %c + r"%d_" %j + r"%d" %i 
#             sv = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AMsnow\C%d_" %c + r"%d_" %j + r"%d" %i
            am1_, am2_, am3_, savg_S1_, savg_S2_, savg_C_ = mf1.AM(ds_rain, conf, c, w)
            am1_.to_pickle(sv + "AM1.pkl")
            am2_.to_pickle(sv + "AM2.pkl")
            am3_.to_pickle(sv + "AM3.pkl")
            savg_S1_.to_pickle(sv + "savg_S1.pkl")
            savg_S2_.to_pickle(sv + "savg_S2.pkl")
            savg_C_.to_pickle(sv + "savg_C.pkl")